In [5]:
# daxpy.py
from typing import Sequence, List
def daxpy(a, x, y):
    if len(x) != len(y):
        raise ValueError("x and y must have the same length")
    return [a*xi + yi for xi, yi in zip(x, y)]


In [6]:
# test_daxpy.py
import math
import random
import pytest
from daxpy import daxpy

def allclose_vec(v1, v2, atol=1e-12, rtol=1e-12):
    if len(v1) != len(v2): 
        return False
    for a, b in zip(v1, v2):
        if math.isfinite(a) and math.isfinite(b):
            if abs(a-b) > atol + rtol*max(abs(a), abs(b)):
                return False
        else:
            # strict handling for non-finite
            if not (math.isnan(a) and math.isnan(b)) and a != b:
                return False
    return True

def test_small_known():
    a = 3.0
    x = [0.1, 0.1, 0.1]
    y = [7.1, 7.1, 7.1]
    assert daxpy(a, x, y) == [7.4, 7.4, 7.4]

def test_zero_scalar():
    assert daxpy(0.0, [1.0, 2.0], [4.0, 5.0]) == [4.0, 5.0]

def test_zero_vectors():
    n = 5
    assert daxpy(2.0, [0.0]*n, [0.0]*n) == [0.0]*n

def test_length_mismatch():
    with pytest.raises(ValueError):
        daxpy(2.0, [1.0, 2.0], [1.0])

def test_random_reproducible():
    rng = random.Random(123)
    n = 1000
    a = 1.7
    x = [rng.uniform(-10, 10) for _ in range(n)]
    y = [rng.uniform(-10, 10) for _ in range(n)]
    ref = [a*xi + yi for xi, yi in zip(x, y)]
    out = daxpy(a, x, y)
    assert allclose_vec(out, ref)

def test_nan_inf_passthrough():
    a = 2.0
    x = [1.0, math.nan, math.inf, -math.inf]
    y = [3.0, 1.0, 5.0, 6.0]
    out = daxpy(a, x, y)   # [5.0, nan, inf, -inf]
    assert math.isclose(out[0], 5.0)
    assert math.isnan(out[1])
    assert math.isinf(out[2]) and out[2] > 0
    assert math.isinf(out[3]) and out[3] < 0

# Optional: compare against NumPy if installed
@pytest.mark.optionalhook
def test_numpy_parity():
    try:
        import numpy as np
    except Exception:
        pytest.skip("NumPy not installed")
    a = 3.0
    x = [0.1, -2.0, 5.5, 0.0]
    y = [7.1,  1.0, 2.5, 8.0]
    out = daxpy(a, x, y)
    ref = (a*np.array(x) + np.array(y)).tolist()
    assert allclose_vec(out, ref)


ModuleNotFoundError: No module named 'daxpy'